# Spotify Analysis

First we install the necessary libraries needed for data visualization and wrangling. Additionally, the R wrapper, Spotifyr, which allows us to get insights from Spotify's web api is installed as well. 

In [73]:
library(tidyverse)
library(spotifyr)
library(repr)
library(lubridate)

Below, we access the web API using the Client ID and Client Secret provided by Spotify upon creating a Developer Account.

In [74]:
access_token <- get_spotify_access_token()

## A Comparison Of My Playlists From Year 1 and Year 2 Of Undergrad
I have attempted to capture each year of my undergraduate degree in a unique playlist. So far, I have one and a half years' worth of data. I will begin my analysis using a playlist I began creating in the fall of 2022, my first year of university. I am interested in learning the general atmosphere of the playlist and how my music preference has changed throughout my first and second years of university. 

### Data Loading & Cleaning

In [75]:
# Loading the playlist data frame into the "playlist_2022" variable

playlist_id <- "2cQLZcVSOLDfY03SY5DZrf"
username <- "AT"
y1_playlist <- get_playlist_audio_features(username, playlist_id)


In [76]:
# Getting all the column names of the data frame
colnames(y1_playlist)

[1] "playlist_id"                        "playlist_name"                     
 [3] "playlist_img"                       "playlist_owner_name"               
 [5] "playlist_owner_id"                  "danceability"                      
 [7] "energy"                             "key"                               
 [9] "loudness"                           "mode"                              
[11] "speechiness"                        "acousticness"                      
[13] "instrumentalness"                   "liveness"                          
[15] "valence"                            "tempo"                             
[17] "track.id"                           "analysis_url"                      
[19] "time_signature"                     "added_at"                          
[21] "is_local"                           "primary_color"                     
[23] "added_by.href"                      "added_by.id"                       
[25] "added_by.type"                      "added_by.uri"                      
[27] "added_by.external_urls.spotify"     "track.artists"                     
[29] "track.available_markets"            "track.disc_number"                 
[31] "track.duration_ms"                  "track.episode"                     
[33] "track.explicit"                     "track.href"                        
[35] "track.is_local"                     "track.name"                        
[37] "track.popularity"                   "track.preview_url"                 
[39] "track.track"                        "track.track_number"                
[41] "track.type"                         "track.uri"                         
[43] "track.album.album_type"             "track.album.artists"               
[45] "track.album.available_markets"      "track.album.href"                  
[47] "track.album.id"                     "track.album.images"                
[49] "track.album.name"                   "track.album.release_date"          
[51] "track.album.release_date_precision" "track.album.total_tracks"          
[53] "track.album.type"                   "track.album.uri"                   
[55] "track.album.external_urls.spotify"  "track.external_ids.isrc"           
[57] "track.external_urls.spotify"        "video_thumbnail.url"               
[59] "key_name"                           "mode_name"                         
[61] "key_mode"

Based on the above columns, some interesting columns to analyze include: 
- **'added_at'**: The date and time the song was added (NA is used if playlist is too old)

- **'track.duration_ms'**: The duration of the track in milliseconds
- **'track.explicit'**: Whether or not the track has explicit lyrics (true = yes it does; false = no it does not OR unknown).
- **'track.name'**: Name of the track
- **'track.popularity'**: A ranking system from 0 to 100, with higher numbers suggesting more popularity. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are.
- **'track.album.name'**: Name of the album
- **'track.album.release_date'**: The date the album was first released
- **'danceability'**: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
- **'energy'**: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
- **'key'**: The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.
- **'track.artists'**: A list of artists who performed the track. Each artist object includes a link to more detailed information about the artist.
- **'loudness'**: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.
- **'mode'**: Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
- **'speechiness'**: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
- **'acousticness'**: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
- **'instrumentalness'**: Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
- **'liveness'**: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.
- **'valence'**: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
- **'tempo'**: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
- **'key_name'**: Key name represented as a string
- **'mode_name'**: Mode represented as a string ("major" or "minor")
- **'key_mode'**: Both key and mode together as a string (ex: "G major")
- **'time_signature'**: An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4".

In [77]:
# Selecting only the identified columns above
# Mutating all date/time columns from character types to date types
shortened_y1_playlist <- y1_playlist  |>
                select(added_at, track.duration_ms, track.explicit, track.name, track.popularity, 
                       track.album.name, track.album.release_date, danceability, energy, key, 
                       track.artists, loudness, mode, speechiness, acousticness, instrumentalness, liveness,
                       valence, tempo, key_name, mode_name, key_mode, time_signature) |>
                mutate(added_at = as_datetime(added_at),
                       track.album.release_date = as_datetime(track.album.release_date))

# Getting the dimensions
dim(shortened_y1_playlist)

# Getting the first six rows of our shorter data frame
head(shortened_y1_playlist)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `track.album.release_date =
  as_datetime(track.album.release_date)`.
Caused by warning:
!  10 failed to parse.”


[1] 567  23

added_at,track.duration_ms,track.explicit,track.name,track.popularity,track.album.name,track.album.release_date,danceability,energy,key,⋯,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key_name,mode_name,key_mode,time_signature
<dttm>,<int>,<lgl>,<chr>,<int>,<chr>,<dttm>,<dbl>,<dbl>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>
2022-08-06 00:01:24,214800,FALSE,Good Looking,1,Good Looking,2017-10-20,0.377,0.558,4,⋯,0.0299,0.0789,3.42e-04,0.1250,0.267,149.971,E,major,E major,3
2022-08-06 00:04:37,202213,FALSE,fOoL fOr YoU,65,Mind Of Mine (Deluxe Edition),2016-03-25,0.520,0.513,7,⋯,0.0256,0.2810,0.00e+00,0.0976,0.170,77.903,G,major,G major,4
2022-08-06 00:17:48,215686,FALSE,Every Summertime,77,Every Summertime,2021-08-10,0.628,0.676,6,⋯,0.0424,0.3930,6.94e-05,0.0978,0.723,78.996,F#,major,F# major,4
2022-08-06 00:18:07,233337,TRUE,Never Be Like You,0,Skin,2016-05-27,0.443,0.558,0,⋯,0.0624,0.4410,0.00e+00,0.1630,0.248,116.838,C,major,C major,4
2022-08-06 00:26:55,164662,TRUE,SOMEBODY,68,GABRIEL,2022-03-25,0.649,0.461,8,⋯,0.0449,0.4420,0.00e+00,0.1080,0.487,141.088,G#,major,G# major,4
2022-08-10 04:30:57,257000,FALSE,My Favourite Clothes,66,My Favourite Clothes,2018-01-06,0.746,0.339,2,⋯,0.1440,0.4820,1.36e-05,0.0838,0.291,119.924,D,major,D major,4


>According to the dimensions of the data frame, there are 567 rows. However, my playlist only has 566 songs! This means there could be some overcounting involved. Let's check if there are duplicate rows:

In [78]:
# Keeping only unique rows from shortened_playlist_2022 and checking if total rows become 566
dim(distinct(shortened_y1_playlist))

[1] 567  23

This shows us that there were no overcounted songs, since even after removing duplicates we are left with 567 rows. To identify the source of the extra row, let's check the total NA counts of each row:

In [79]:
# Arranging total occurances of NA's per row in descending order
na_per_rows <- as_tibble(rowSums(is.na(shortened_y1_playlist))) |>
            rename(na_count = value)|> arrange(- na_count)
# Showing the 6 highest NA counts
head(na_per_rows)

na_count
<dbl>
20
1
1
1
1
1


Above we can see that one row in particular has 20 NA values. This is quite unusual as the highest NA values all other rows have is 1. This seems to be the row that caused the extra row count in our playlist data frame. Perhaps Spotify puts NA for songs that have been deleted from their platform. Since there is no data in the row, we can safely remove this row from our playlist data.

In [82]:
shortened_y1_playlist <- shortened_y1_playlist[rowSums(is.na(shortened_y1_playlist)) != 20,]
dim(shortened_y1_playlist)

[1] 566  23

I will now also load my year 2 playlist which has 101 songs in it:

In [81]:
playlist_id <- "6paPUoyazZ2SkgZ5HkYpoW"
shortened_y2_playlist <- get_playlist_audio_features(username, playlist_id) |>
                select(added_at, track.duration_ms, track.explicit, track.name, track.popularity, 
                       track.album.name, track.album.release_date, danceability, energy, key, 
                       track.artists, loudness, mode, speechiness, acousticness, instrumentalness, liveness,
                       valence, tempo, key_name, mode_name, key_mode, time_signature) |>
                 mutate(added_at = as_datetime(added_at),
                        track.album.release_date = as_datetime(track.album.release_date))

# Getting dimensions and last 6 rows of the y2 playlist
dim(shortened_y2_playlist)
tail(shortened_y2_playlist)


[1] 101  23

added_at,track.duration_ms,track.explicit,track.name,track.popularity,track.album.name,track.album.release_date,danceability,energy,key,⋯,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key_name,mode_name,key_mode,time_signature
<dttm>,<int>,<lgl>,<chr>,<int>,<chr>,<dttm>,<dbl>,<dbl>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>
2023-12-10 18:03:34,215555,FALSE,Listerine,59,Fuzzybrain,2019-11-14,0.694,0.583,11,⋯,0.0371,0.1700,8.20e-01,0.3290,0.897,134.999,B,major,B major,4
2023-12-10 18:03:48,201857,FALSE,out 4 a breather,46,I AM JUST A BOY LEAVE ME ALONE !!!,2021-03-19,0.680,0.529,5,⋯,0.0285,0.3310,1.46e-02,0.1370,0.690,127.024,F,major,F major,4
2023-12-15 13:29:11,278906,TRUE,Trouble In Town,49,Everyday Life,2019-11-22,0.595,0.315,2,⋯,0.0296,0.4270,6.48e-01,0.1110,0.336,96.018,D,minor,D minor,4
2023-12-21 15:40:15,200961,FALSE,Fiona,41,What's Your Motive,2023-09-08,0.764,0.588,10,⋯,0.0343,0.0569,2.57e-06,0.0519,0.877,112.011,A#,minor,A# minor,4
2023-12-22 03:36:04,231680,TRUE,Juno,52,Honeybloom,2018-08-03,0.438,0.266,7,⋯,0.0363,0.9220,2.98e-04,0.1480,0.116,120.075,G,major,G major,4
2024-01-03 00:22:13,205554,FALSE,I Hope to Be Around,59,I Hope to Be Around,2017-11-10,0.821,0.384,11,⋯,0.0388,0.7800,1.38e-03,0.1100,0.790,87.978,B,major,B major,4


### A Visual Analysis